In [1]:
# Package imports
import pandas as pd

In [8]:
# Load cleaned datasets
dispatch_data = pd.read_csv("data/cleaned_dispatch_data.csv")
traffic_data = pd.read_csv("data/cleaned_traffic_data.csv")
weather_data = pd.read_csv("data/cleaned_weather_data.csv")
left_merged_data = pd.read_csv('data/left_merged_data.csv')


# Create features

In [9]:
# Add hours features
left_merged_data['dispatch_hour'] = pd.to_datetime(left_merged_data['dispatch_time']).dt.hour
left_merged_data['dispatch_day'] = pd.to_datetime(left_merged_data['dispatch_time']).dt.day

# Category encoding
left_merged_data['job_type_encoded'] = left_merged_data['job_type'].astype('category').cat.codes
left_merged_data['traffic_level_encoded'] = left_merged_data['traffic_level'].map({'low': 1, 'moderate': 2, 'high': 3, 'severe': 4})

# Normalize/Standardize columns
left_merged_data['temperature_normalized'] = (left_merged_data['temperature'] - left_merged_data['temperature'].mean()) / left_merged_data['temperature'].std()
left_merged_data['precipitation_normalized'] = (left_merged_data['precipitation'] - left_merged_data['precipitation'].mean()) / left_merged_data['precipitation'].std()

In [10]:
### Fill NA's with values ###

# Fill missing traffic levels with mode
left_merged_data['traffic_level'].fillna('moderate', inplace=True)

# Fill missing weather data with mean values
left_merged_data['temperature'].fillna(left_merged_data['temperature'].mean(), inplace=True)
left_merged_data['precipitation'].fillna(left_merged_data['precipitation'].mean(), inplace=True)

In [11]:
# Find what % of rows are from all data sources and label them
# Data in all sources?
left_merged_data['full_data'] = (
    (left_merged_data['source_dispatch'] == 1) &
    (left_merged_data['source_traffic'] == 1) &
    (left_merged_data['source_weather'] == 1)
)

left_merged_data['full_data'].value_counts().reset_index(name ='counts')

,full_data,counts
0,True,499
1,False,1


In [12]:
# Filter rows with partial data
partial_data = left_merged_data[left_merged_data['full_data'] == False]

# Find whether any particular data source is missing more often than others, then why?
missing_source_counts = partial_data[['source_dispatch', 'source_traffic', 'source_weather']].sum()
print(missing_source_counts)

# Are there specific days missing or specific locations missing too often?
missing_weather = partial_data[partial_data['source_weather'] == 0]
print(missing_weather[['location_id', 'date']].head())


# Find which location_id values are missing in weather_data
missing_location_ids = missing_weather['location_id'].unique()
print("Missing location_ids:", missing_location_ids)

# Check if these location_ids exist in weather_data
missing_in_weather_data = weather_data[weather_data['location_id'].isin(missing_location_ids)]
print("Weather data for missing location_ids:\n", missing_in_weather_data)

source_dispatch    1
source_traffic     0
source_weather     0
dtype: int64
    location_id        date
40          109  2024-01-10
Missing location_ids: [109]
Weather data for missing location_ids:
     location_id        date  temperature  precipitation  source_weather
24          109  2024-01-01         60.9           0.90               1
25          109  2024-01-02         65.3           0.08               1
26          109  2024-01-03         65.6           0.44               1


In [13]:
# Example: Save after dropping partial rows or imputing
final_data = left_merged_data[left_merged_data['full_data'] == True].copy()
# OR some function that does advanced imputation

final_data.to_csv("data/final_modeling_data.csv", index=False)